In [1]:
import copy
import pickle

import jax.nn
import jax.random as random
import matplotlib.pyplot as plt
import numpyro
from numpyro import handlers

In [2]:
%load_ext autoreload
%autoreload 2

from experiments.src.experiment import *
from experiments.src.data import ToyData1
from experiments.src.model import BNNRegressor

In [3]:
# %matplotlib inline
# matplotlib.use("nbAgg")  # noqa: E402

plt.rcParams.update({
    "axes.grid": True,      # show grid by default
    "font.weight": "bold",  # bold fonts
    "xtick.labelsize": 15,  # large tick labels
    "ytick.labelsize": 15,  # large tick labels
    "lines.linewidth": 1,   # thick lines
    "lines.color": "k",     # black lines
    # "grid.color": "0.5",    # gray gridlines
    "grid.linestyle": "-",  # solid gridlines
    "grid.linewidth": 0.1,  # thin gridlines
    "savefig.dpi": 300,     # higher resolution output.
})

In [4]:
DEVICE = "cpu"
numpyro.set_platform(DEVICE)
# numpyro.set_host_device_count(NUM_CHAINS)
D_X = 3
VI_MAX_ITER = 100_000
BNN_SIZE = [32, 32, 16]
BETA = 1.0
# FIG_PREFIX = f"fig"

## Data

In [5]:
data = ToyData1(D_X=D_X, train_size=100)

## Model

In [6]:
bnn = BNNRegressor(
    nonlin=jax.nn.silu,
    D_X=D_X,
    D_Y=1,
    D_H=BNN_SIZE,
    biases=True,
    prior_scale=10,
    prior_type='xavier',
    obs_model='loc_scale',
    # obs_model=1 / (0.05 / 0.26480442)**2,
    beta=BETA,
)


In [7]:
bnn.get_weight_dim()

1746

In [8]:
fresh_bnn = copy.deepcopy(bnn)

## Experiment

In [9]:
experiment = BasicHMCExperiment(
    bnn,
    data,
    num_samples = 60, num_warmup = 30,
    # num_chains = 4, group_by_chain=True
)
experiment.run(random.PRNGKey(0))#.savefig("figs/simple_hmc_4.png")

  0%|          | 0/90 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,10), sharex=True, sharey=True)
# for i, ax in enumerate(axs.ravel()):
#     ax.plot(data.test[0][:, 1], experiment._predictions["Y_mean"][..., 0][i].mean(axis=0))
#     ax.fill_between(data.test[0][:, 1], *np.percentile(experiment._predictions["Y_mean"][..., 0][i], (5.0, 95.0), axis=0), alpha=0.5, color="orange")
#     ax.fill_between(data.test[0][:, 1], *np.percentile(experiment._predictions["Y"][..., 0][i], (5.0, 95.0), axis=0), alpha=0.5, color="lightgreen")
#     ax.plot(data.train[0][:, 1], data.train[1], "kx")
#     ax.set_ylim(-6, +6)
# fig.tight_layout()
# fig.savefig("figs/hmc-by-chain.png")

In [ ]:
experiment = AutoLaplaceExperiment(
    fresh_bnn,
    data,
    num_samples=10_000,
    max_iter=VI_MAX_ITER,
    diag=True,
    shrink=100.0,
)
rk = random.PRNGKey(0)
# experiment.run(random.PRNGKey(1))
# experiment.show_convergence_plot()
# experiment.run(random.PRNGKey(1)).savefig("figs/simple_mfvi_4.png")
# experiment.show_convergence_plot().savefig("figs/simple_mfvi_conv_4.png")

In [ ]:
assert False
for i in tqdm.tqdm(range(15)):
    rk, rkc = random.split(rk)
    experiment.train(rkc)
    with open("svi-state.pkl", "wb") as f:
        pickle.dump(experiment._saved_svi_state, f)
    rk, rkc = random.split(rk)
    experiment.make_predictions(rkc)
    experiment.make_plots().savefig(f"figs/post-pred-{i}.png")
    fig, ax = plt.subplots()
    # ax.plot(experiment._losses[::(VI_MAX_ITER//50)])
    ax.plot(experiment._eval_losses)
    ax.set_ylim(experiment._eval_losses.min(), jnp.percentile(experiment._eval_losses, 90.0))
    fig.savefig(f"figs/loss-{i}.png")

In [ ]:
# with open("/tmp/svi-state.pkl", "rb") as f:
#     print(pickle.load(f))

In [ ]:
# go again:
rk, rkc = random.split(rk)
experiment.train(rkc, num_iter=10_000)

In [ ]:
rk, rkc = random.split(rk)
experiment.make_predictions(rkc)
experiment.make_plots().show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(-experiment._eval_losses[:, 1], label="-loglik")
ax.plot(experiment._eval_losses[:, 2], label="+kl")
ax.legend()
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(experiment._losses[1000::10*(VI_MAX_ITER // VI_MAX_ITER)])
# ax.plot(experiment._eval_losses)
ax.set_ylim(0, 1000)
fig.show()

### Loglikelihood analysis

In [ ]:
def sample_posterior(rng_key):
    fitted_guide = handlers.substitute(handlers.seed(experiment._get_guide(), rng_key), experiment._params)
    trace = handlers.trace(fitted_guide).get_trace(X=experiment._data.train[0], Y=None)
    # print(numpyro.util.format_shapes(trace))
    return dict(w=trace["w"]["value"])

In [ ]:
n = 6
post_fns = vmap(sample_posterior)(random.split(random.PRNGKey(0), n))

In [ ]:
def loglik(rng_key, params):
    rng_lik, rng_gen = random.split(rng_key)
    model = handlers.substitute(handlers.seed(experiment._bnn, rng_lik), params)
    trace = handlers.trace(model).get_trace(X=experiment._data.train[0], Y=experiment._data.train[1])
    # print(numpyro.util.format_shapes(trace, compute_log_prob=True))
    w_node = trace["w"]
    y_node = trace["Y"]

    model = handlers.substitute(handlers.seed(experiment._bnn, rng_gen), params)
    trace =  handlers.trace(model).get_trace(X=experiment._data.test[0], Y=None)
    return dict(prior_logprob=w_node['fn'].log_prob(w_node['value']),
                loglik=y_node['fn'].log_prob(y_node['value']).sum(),
                Y_mean=trace["Y_mean"]["value"].squeeze(),
                Y_scale=trace["Y_scale"]["value"].squeeze())

In [ ]:
ll = vmap(loglik)(random.split(random.PRNGKey(1), n), post_fns)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(9, 6))
for i, ax in enumerate(axs.ravel()):
    ax.plot(data.train[0][:, 1], data.train[1][:, 0], 'kx', alpha=0.2)
    ax.plot(data.test[0][:, 1], ll['Y_mean'][i])
    ax.fill_between(data.test[0][:, 1], ll['Y_mean'][i] - ll['Y_scale'][i] * 2,
                    ll['Y_mean'][i] + ll['Y_scale'][i] * 2, alpha=0.2)
    ax.set_title(f"loglik={ll['loglik'][i]:.3f}\nlogprior={ll['prior_logprob'][i]:.3f}")
fig.tight_layout()
fig.show()

In [ ]:
# fig, ax = plt.subplots()
# for i in range(10):
#     ax.plot(fresh_data.test[0][:, 1], preds[i])
# plt.show()

In [ ]:
# fig.savefig("figs/posterior_predictive_1.png")

### Sequential experiment

## Space for running experiments

In [ ]:
tr = handlers.trace(handlers.seed(bnn, random.PRNGKey(0))).get_trace(X=data.train[0], Y=None)
print(numpyro.util.format_shapes(tr, compute_log_prob=True))

In [ ]:
# fresh_bnn = copy.deepcopy(bnn)
fresh_data = copy.deepcopy(data)
first_half_data = DataSlice(fresh_data, slice(50))
second_half_data = DataSlice(fresh_data, slice(50,100))

In [ ]:
first_half_experiment = BasicMeanFieldGaussianVIExperiment(fresh_bnn, first_half_data, num_samples=1000, max_iter=VI_MAX_ITER)
first_half_experiment.run(random.PRNGKey(0)).savefig("figs/manual_first_half_VI_4.png")
first_half_experiment.show_convergence_plot().savefig("figs/manual_first_half_VI_conv_4.png")

In [ ]:
# rk = random.PRNGKey(0)
# # For consistency with exp
# rk, _ = random.split(rk)

In [ ]:
# first_half_experiment._data.train[0][..., 1].max()

In [ ]:
# rk, rkc = random.split(rk)
# first_half_experiment.train(rkc, num_iter=1_000)

In [ ]:
# rk, rkc = random.split(rk)
# first_half_experiment.make_predictions(rkc)
# first_half_experiment.make_plots().show()

In [ ]:
# plt.plot(first_half_experiment._losses[::(VI_MAX_ITER // 50)])
# plt.plot(first_half_experiment._eval_losses)

In [ ]:
second_half_HMC_experiment = BasicHMCExperiment(fresh_bnn.with_prior(*first_half_experiment.posterior), second_half_data, num_samples=1000, num_warmup=500)
second_half_HMC_experiment.run(random.PRNGKey(0)).savefig("figs/manual_second_half_HMC_4.png")

In [ ]:
second_half_VI_experiment = BasicMeanFieldGaussianVIExperiment(fresh_bnn.with_prior(*first_half_experiment.posterior), second_half_data, num_samples=1000, max_iter=5*VI_MAX_ITER)
second_half_VI_experiment.run(random.PRNGKey(0)).savefig("figs/manual_second_half_VI_4.png")

In [ ]:
# fresh_data = copy.deepcopy(data)

In [ ]:
sequential_experiment = SequentialExperiment(fresh_bnn, fresh_data, BasicMeanFieldGaussianVIExperiment, num_inference_steps=2, max_iter = VI_MAX_ITER)

In [ ]:
sequential_experiment = ExperimentWithLastBlockReplaced(sequential_experiment, BasicHMCExperiment, num_samples=200, num_warmup=100)

In [ ]:
# sequential_experiment.train(random.PRNGKey(0))

In [ ]:
# sequential_experiment.make_predictions(random.PRNGKey(2), final_only=False)

In [ ]:
# sequential_experiment.make_plots(final_only=False)

In [ ]:
# # Custom plotting for sequential experiment
# fig, axs = plt.subplots(figsize=(8, 4), ncols=2)
# for i, ax in enumerate(axs.ravel()):
#     experiment_block = sequential_experiment._sequential_experiment._experiment_blocks[i]
#     predictions = experiment_block._predictions["Y"][..., 0]
#     mean_predictions = experiment_block._predictions["Y_mean"][..., 0]
#     data = experiment_block._data
#     X, Y = data.train
#     X_test, _ = data.test
#     # compute mean prediction and confidence interval around median
#     mean_means = jnp.mean(mean_predictions, axis=0)
#     mean_percentiles = np.percentile(predictions, [5.0, 95.0], axis=0)
#     # plot training data
#     ax.plot(X[:, 1], Y[:, 0], "kx")
#     # plot predictions & quantiles
#     ax.plot(X_test[:, 1], mean_means, color="blue")
#     ax.fill_between(X_test[:, 1], *mean_percentiles, color="lightblue")
#     ax.set_title(str(data._train_idx_slice))
# fig.tight_layout()
# fig.savefig("figs/sequential-VI-simple3.png")

#### Draw samples from prior predictive

In [ ]:
fig, ax =  plt.subplots()
with handlers.seed(rng_seed=random.PRNGKey(1)):
    t = data.test[0]
    for _ in range(50):
        prior_fn = handlers.trace(fresh_bnn).get_trace(X=t, Y=None)["Y_mean"]["value"]
        ax.plot(t[:,1], prior_fn, alpha=0.5)
        # ax.set_yscale('log')
# plt.savefig("figs/prior_pred_4.png")

#### Reverse dataset for two halves sequential experiment

In [ ]:
# fresh_bnn = copy.deepcopy(bnn)

In [ ]:
reversed_data = ReverseData(data)

In [ ]:
reversed_sequential_experiment = SequentialExperiment(fresh_bnn, reversed_data, BasicFullRankGaussianVIExperiment)

In [ ]:
# reversed_sequential_experiment.train(random.PRNGKey(0))

In [ ]:
# reversed_sequential_experiment.make_predictions(random.PRNGKey(1), final_only=False)

In [ ]:
# # Custom plotting for reversed sequential experiment
# fig, axs = plt.subplots(figsize=(8, 4), ncols=2)
# for i, ax in enumerate(axs.ravel()):
#     experiment_block = reversed_sequential_experiment._experiment_blocks[i]
#     predictions = experiment_block._predictions["Y"][..., 0]
#     data = experiment_block._data
#     X, Y = data.train
#     X_test, _ = data.test
#     # compute mean prediction and confidence interval around median
#     mean_predictions = jnp.mean(predictions, axis=0)
#     percentiles = np.percentile(predictions, [5.0, 95.0], axis=0)
#     # plot training data
#     ax.plot(X[:, 1], Y[:, 0], "kx")
#     # plot predictions & quantiles
#     ax.plot(X_test[:, 1], mean_predictions, color="blue")
#     ax.fill_between(X_test[:, 1], *percentiles, color="lightblue")
#     ax.set_title(str(data._train_idx_slice))
# fig.tight_layout()
# # fig.savefig("figs/sequential-full-rank-VI-reversed1.png")

#### Randomized dataset experiment

In [ ]:
train_len = data.train[0].shape[0]
random_perm = np.random.choice(np.arange(train_len), size=train_len, replace=False)
permuted_data = PermutedData(data, random_perm)

In [ ]:
plt.close('all')